## Import Libraries

In [10]:
import pandas as pd
import pickle
import re
import numpy as np
import os

## Read CSV files and merge

In [11]:
pd.options.mode.chained_assignment = None
raw_dataset_dir = "../Raw_Data/"
house_dataset = []
files = [file for file in os.listdir(raw_dataset_dir) if 'csv' in file]
for idx in range(len(files)):
    house_dataset.append(pd.read_csv(raw_dataset_dir + files[idx], index_col = [0]))
dataset = pd.concat(house_dataset, ignore_index=True)
print(dataset.shape)
dataset.head()

(4561, 3)


,Location,Price,Area
0,"380 Phố Tôn Đức Thắng, Phường Hòa Minh, Liên C...",2 triệu/tháng,35 m2
1,"346 Đường Ngũ Hành Sơn, Phường Mỹ An, Ngũ Hành...",2.5 triệu/tháng,35 m2
2,"582 Đường số 2/9, Phường Hòa Cường Nam, Hải Ch...",2 triệu/tháng,35 m2
3,"559 Trần Cao Vân Đường Trần Cao Vân, Phường Xu...",1.7 triệu/tháng,20 m2
4,"Phường Hòa Cường Bắc, Hải Châu, Đà Nẵng",2.2 triệu/tháng,18 m2


## Remove extraordinary data and Modify data

In [12]:
for id in range(len(dataset)):
    try:
        dataset['Area'][id] = dataset['Area'][id].split(": ")[1]
    except: pass
    try:
        dataset['Location'][id] = dataset['Location'][id].replace('Phường ', '').replace('P. ', '').replace('Quận ', '').replace('Q. ', '').replace('  ', ' ')
    except: pass
    if(('PN' in dataset['Area'][id]) or ('Tỷ' in dataset['Price'][id]) or ('tỷ' in dataset['Price'][id]) or ('Thương lượng' in dataset['Price'][id]) or ('Thỏa thuận' in dataset['Price'][id])):
        dataset = dataset.drop(index=id)
    
dataset = dataset.reset_index(drop = True)
dataset

,Location,Price,Area
0,"380 Phố Tôn Đức Thắng, Hòa Minh, Liên Chiểu, Đ...",2 triệu/tháng,35 m2
1,"346 Đường Ngũ Hành Sơn, Mỹ An, Ngũ Hành Sơn, Đ...",2.5 triệu/tháng,35 m2
2,"582 Đường số 2/9, Hòa Cường Nam, Hải Châu, Đà ...",2 triệu/tháng,35 m2
3,"559 Trần Cao Vân Đường Trần Cao Vân, Xuân Hà, ...",1.7 triệu/tháng,20 m2
4,"Hòa Cường Bắc, Hải Châu, Đà Nẵng",2.2 triệu/tháng,18 m2
...,...,...,...
4347,Núi Thành,"4,8 triệu/tháng",30 m²
4348,"Núi Thành, Hòa Cường Bắc, Hải Châu, Đà Nẵng","4,8 triệu/tháng",30 m²
4349,"148, Đường Ngũ Hành Sơn, Mỹ An, Ngũ Hành Sơn, ...","1,7 triệu/tháng",20 m²
4350,"Đường Tiên Sơn 6, Hòa Cường Nam, Hải Châu, Đà ...",5 triệu/tháng,40 m²


## Extract useful information

### Extract Ward and District in Location 

In [13]:
#List of district in Danang
Districts = ['Hải Châu', 'Cẩm Lệ', 'Liên Chiểu', 'Ngũ Hành Sơn', 'Sơn Trà', 'Hòa Vang', 'Thanh Khê', 'Hoàng Sa']
#List of ward in Danang
Wards = ['Hòa An', 'Hòa Phát', 'Hòa Thọ Đông', 'Hòa Thọ Tây', 'Hòa Xuân', 'Khuê Trung', 'Bình Hiên', 'Bình Thuận', 'Hải Châu I', 'Hải Châu II', 'Hòa Cường Bắc', 'Hòa Cường Nam', 'Hòa Thuận Đông', 'Hòa Thuận Tây', 'Nam Dương', 'Phước Ninh', 'Thạch Thang', 'Thanh Bình', 'Thuận Phước', 'Hòa Hiệp Bắc', 'Hòa Hiệp Nam', 'Hòa Khánh Bắc', 'Hòa Khánh Nam', 'Hòa Minh', 'Hòa Hải', 'Hòa Quý', 'Khuê Mỹ', 'Mỹ An', 'An Hải Bắc', 'An Hải Đông', 'An Hải Tây', 'Mân Thái', 'Nại Hiên Đông', 'Phước Mỹ', 'Thọ Quang', 'An Khê', 'Chính Gián', 'Hòa Khê', 'Tam Thuận', 'Tân Chính', 'Thạc Gián', 'Thanh Khê Đông', 'Thanh Khê Tây', 'Vĩnh Trung', 'Xuân Hà', 'Hòa Phong', 'Hòa Bắc', 'Hòa Châu', 'Hòa Khương', 'Hòa Liên', 'Hòa Nhơn', 'Hòa Ninh', 'Hòa Phú', 'Hòa Phước', 'Hòa Sơn', 'Hòa Tiến']

dataset['Ward'] = 'Unknown W'
dataset['District'] = 'Unknown D'

for i in range(len(dataset['Location'])):
    location_components = dataset['Location'][i].split(", ")
    for ward in Wards:
        if ward in location_components:
            dataset['Ward'][i] = ward
    for district in Districts:
        if district in location_components:
            dataset['District'][i] = district
    if (dataset['District'][i] ==  'Unknown D'):
        dataset = dataset.drop(index=i)  

dataset = dataset.reset_index(drop = True).drop(columns='Location')
dataset

,Price,Area,Ward,District
0,2 triệu/tháng,35 m2,Hòa Minh,Liên Chiểu
1,2.5 triệu/tháng,35 m2,Mỹ An,Ngũ Hành Sơn
2,2 triệu/tháng,35 m2,Hòa Cường Nam,Hải Châu
3,1.7 triệu/tháng,20 m2,Xuân Hà,Thanh Khê
4,2.2 triệu/tháng,18 m2,Hòa Cường Bắc,Hải Châu
...,...,...,...,...
4246,1 triệu/tháng,16 m²,Hòa Khánh Bắc,Liên Chiểu
4247,"4,8 triệu/tháng",30 m²,Hòa Cường Bắc,Hải Châu
4248,"1,7 triệu/tháng",20 m²,Mỹ An,Ngũ Hành Sơn
4249,5 triệu/tháng,40 m²,Hòa Cường Nam,Hải Châu


### Extract area value in Area 

In [14]:
#Regex for extracting useful information
area_re = "\d+"

dataset['Area'] = dataset['Area'].apply(lambda x: re.findall(area_re, x)[0])
for idx in range(len(dataset['Area'])):
    dataset['Area'][idx] = int(dataset['Area'][idx])
dataset

,Price,Area,Ward,District
0,2 triệu/tháng,35,Hòa Minh,Liên Chiểu
1,2.5 triệu/tháng,35,Mỹ An,Ngũ Hành Sơn
2,2 triệu/tháng,35,Hòa Cường Nam,Hải Châu
3,1.7 triệu/tháng,20,Xuân Hà,Thanh Khê
4,2.2 triệu/tháng,18,Hòa Cường Bắc,Hải Châu
...,...,...,...,...
4246,1 triệu/tháng,16,Hòa Khánh Bắc,Liên Chiểu
4247,"4,8 triệu/tháng",30,Hòa Cường Bắc,Hải Châu
4248,"1,7 triệu/tháng",20,Mỹ An,Ngũ Hành Sơn
4249,5 triệu/tháng,40,Hòa Cường Nam,Hải Châu


### Extract price value in Price

In [15]:
#Regex for extracting useful information
price_re = "([(\d\W)*\d+]+(Triệu|triệu|Đồng|đ|Nghìn)+)?"
price_stn = "(\d+)\W*(\d*) (Triệu|triệu|Đồng|đ|Nghìn)"

m_price = 1000000
dataset['Price'] = dataset['Price'].apply(lambda x: re.match(price_re, x)[0])
for i in range(len(dataset['Price'])):
    try:
        price_components = list(re.findall(price_stn, dataset['Price'][i])[0])
        if (price_components[2] == 'Triệu' or price_components[2] == 'triệu'):
            if (price_components[1] != ''): # 2 & 5 & trieu
                dataset['Price'][i] = int(float(price_components[0]) * m_price + float(str(0.) + price_components[1]) * 10 * m_price)
            else:
                dataset['Price'][i] = int(float(price_components[0]) * m_price)

        else:
            dataset['Price'][i] = int(float(price_components[0]) * 1000)
    except: dataset['Price'][i] = np.Nan

dataset

AttributeError: module 'numpy' has no attribute 'Nan'

Price       0
Area        0
Ward        0
District    0
dtype: int64

## Save dataset to csv file

In [ ]:
dataset.to_csv('../Dataset/dataset.csv')